In [6]:
import os
from pathlib import Path
from typing import Callable, Optional, Union, List, Tuple
import h5py
import nibabel as nib
import numpy as np
import fastmri.data.utils as utils
import pandas as pd

def filter_data(dataset: pd.DataFrame, root: Union[str, Path], data_partition: str) -> List[Path]:
    """
    根据数据分区过滤文件列表。

    Args:
        dataset: 包含数据文件和分区信息的 pandas DataFrame。
        root: 数据集的根目录。
        data_partition: 数据分区，"train"、"val"或"test"。
    
    Returns:
        对应数据分区的文件路径列表。
    """
    # 使用 'Split' 列过滤属于指定数据分区的文件
    filtered_files = dataset[dataset['Split'] == data_partition]['Name']

    # 将文件名转换为完整路径并加上扩展名
    file_list = [Path(root) / (fname + '.h5') for fname in filtered_files]

    return file_list

def get_file_list(root: Union[str, Path, os.PathLike], data_partition: str) -> List[Path]:
    """
    获取指定数据分区的文件列表。

    Args:
        root: 数据集的根路径。
        data_partition: 数据分区，"train"、"val"或"test"。
    
    Returns:
        文件路径的列表。
    """
    root = Path(root)  # 将 root 转换为 Path 对象

    # 假设你的 dataset.csv 文件在 root 目录中
    dataset = pd.read_csv("/Users/ziling/Desktop/MRCP/MRCP_DLRecon-main/sample_data/dataset.csv")

    # 根据 data_partition 过滤文件列表，假设 filter_data 是你用来过滤数据的函数
    file_list = filter_data(dataset, root, data_partition)

    return file_list


def retrieve_metadata(fname: Union[str, Path, os.PathLike]) -> int:
    """
    获取文件的切片数量。
    
    Args:
        fname: 文件路径。
    
    Returns:
        文件中的切片数量。
    """
    with h5py.File(fname, "r") as hf:
        num_slices = hf["kdata"].shape[-1]
    return num_slices


def load_slice_data(fname: Union[str, Path, os.PathLike], dataslice: int) -> Tuple:
    """
    从文件中加载指定切片的数据，包括k空间、重建结果和敏感度图。

    Args:
        fname: 文件路径。
        dataslice: 要加载的切片索引。
    
    Returns:
        k空间数据、GRAPPA重建结果、敏感度图、文件属性和基础加速因子。
    """
    with h5py.File(fname, "r") as hf:
        kspace = hf["kdata"][..., dataslice]  # k空间数据
        target = hf["grappa"][..., dataslice]  # GRAPPA重建结果
        sens_maps = hf["sm_espirit"][..., dataslice]  # 敏感度图
        
        attrs = dict(hf.attrs)
        base_acc = attrs.get('base_acc', 1)  # 基础加速因子，默认为1（可以是2或6）
        
    return kspace, target, sens_maps, base_acc, attrs


def create_examples(file_list: List[Path]) -> List[Tuple[Path, int]]:
    """
    为每个文件生成切片示例。
    
    Args:
        file_list: 文件列表。
    
    Returns:
        文件和对应的切片索引的列表。
    """
    examples = []
    for fname in sorted(file_list):
        num_slices = retrieve_metadata(fname)
        examples += [(fname, slice_idx) for slice_idx in range(num_slices)]
    return examples


def load_dataset(root: Union[str, Path, os.PathLike], data_partition: str = "train", is_prototype: bool = False) -> List[Tuple[Path, int]]:
    """
    加载数据集，生成所有文件及其切片的列表。

    Args:
        root: 数据集的根目录。
        data_partition: 数据分区，默认为 "train"。
        is_prototype: 是否只加载少量数据用于调试。

    Returns:
        文件和切片索引的列表。
    """
    files = get_file_list(root, data_partition)

    # 如果是原型模式，则只使用部分数据
    if is_prototype:
        files = files[:1]  # 调试时只使用第一个文件

    examples = create_examples(files)
    return examples


def convert_to_nifti(kspace: np.ndarray, target: np.ndarray, sens_maps: np.ndarray, output_dir: Union[str, Path], file_info: List[str]):
    """
    将从 h5 文件中提取的 MRI 数据转换为 NIfTI 格式并保存到指定目录。

    Args:
        kspace: 原始的 k-space 数据。
        target: 重建后的图像（如 GRAPPA）。
        sens_maps: 敏感度图数据。
        output_dir: 保存 NIfTI 文件的目录。
        file_info: 文件信息，包括文件名和切片号。
    
    Returns:
        None
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)  # 确保输出目录存在

    # 生成文件名
    file_name = f"{file_info[0]}_slice_{file_info[1]}.nii.gz"

    # 将 GRAPPA重建图像保存为 NIfTI 格式
    nii_img = nib.Nifti1Image(target, affine=np.eye(4))  # 假设 affine 矩阵为单位矩阵，可以根据需要修改
    output_path = output_dir / file_name
    nib.save(nii_img, str(output_path))

    print(f"Saved NIfTI file: {output_path}")


def transform(kspace, target, target_acc, base_acc, fname, dataslice, sens_maps, dinfo):
    """
    数据转换函数，将 h5 数据提取并转换为 NIfTI 格式。
    
    Args:
        kspace: k-space 数据。
        target: 重建后的图像数据。
        target_acc: 目标加速因子。
        base_acc: 基础加速因子。
        fname: 文件名。
        dataslice: 切片编号。
        sens_maps: 敏感度图。
        dinfo: 文件信息，包含文件名和文件夹名。
    
    Returns:
        None
    """
    output_dir = "/Users/ziling/Desktop/MRCP/public_dataset/"  # 设定你想保存NIfTI文件的目录

    # 调用转换函数，将数据保存为NIfTI格式
    convert_to_nifti(kspace, target, sens_maps, output_dir, dinfo)
    
    # 可以返回其他内容作为必要的信息，如果需要处理其他部分
    return {"filename": fname, "slice": dataslice}


def get_sample(examples: List[Tuple[Path, int]], idx: int, transform: Callable, target_acc: int) -> dict:
    """
    获取指定索引的数据样本。

    Args:
        examples: 文件和切片的列表。
        idx: 样本索引。
        transform: 数据变换函数。
        target_acc: 目标加速因子。
    
    Returns:
        经过变换后的样本。
    """
    fname, dataslice = examples[idx]
    kspace, target, sens_maps, base_acc, attrs = load_slice_data(fname, dataslice)

    dinfo = [f"{fname.stem}_{dataslice}", fname.parent.stem]  # 文件和切片信息
    sample = transform(kspace, target, target_acc, base_acc, fname.name, dataslice, sens_maps, dinfo)

    return sample

def get_samples(examples: List[Tuple[Path, int]], indices: List[int], transform: Callable, target_acc: int) -> List[dict]:
    """
    获取指定索引的多个数据样本。

    Args:
        examples: 文件和切片的列表。
        indices: 样本索引的列表（可以是多个索引）。
        transform: 数据变换函数。
        target_acc: 目标加速因子。
    
    Returns:
        经过变换后的样本的列表。
    """
    samples = []
    for idx in indices:
        fname, dataslice = examples[idx]
        kspace, target, sens_maps, base_acc, attrs = load_slice_data(fname, dataslice)

        dinfo = [f"{fname.stem}_{dataslice}", fname.parent.stem]  # 文件和切片信息
        sample = transform(kspace, target, target_acc, base_acc, fname.name, dataslice, sens_maps, dinfo)
        samples.append(sample)
    
    return samples

# 示例使用
root_dir = "/Users/ziling/Desktop/MRCP/public_dataset/"
data_partition = "train"
target_acc = 6
is_prototype = False

# 加载数据集
examples = load_dataset(root_dir, data_partition, is_prototype)
# 获取并转换数据集中的多个样本
sample_indices = [0, 1, 2]  # 获取前三个样本（假设存在多个样本）
samples = get_samples(examples, sample_indices, transform, target_acc)

# 打印样本信息
for sample in samples:
    print(f"Sample: {sample['filename']}, Slice: {sample['slice']}")

Saved NIfTI file: /Users/ziling/Desktop/MRCP/public_dataset/data1_0_slice_public_dataset.nii.gz
Saved NIfTI file: /Users/ziling/Desktop/MRCP/public_dataset/data1_1_slice_public_dataset.nii.gz
Saved NIfTI file: /Users/ziling/Desktop/MRCP/public_dataset/data1_2_slice_public_dataset.nii.gz
Sample: data1.h5, Slice: 0
Sample: data1.h5, Slice: 1
Sample: data1.h5, Slice: 2
